In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('drive/My Drive/DataDays/second_round')

In [0]:
df = pd.read_csv('divar_posts_dataset.csv')

In [0]:
df['cat1_2'] = df.cat1.astype(str).str.cat(df.cat2.astype(str), sep='|')
df['cat1_2_3'] = df.cat1_2.astype(str).str.cat(df.cat3.astype(str), sep='|')

In [0]:
train,test = train_test_split(df)

In [0]:
labels = {j:i for i,j in enumerate(train.cat1_2_3.unique())}

In [0]:
X_train = train.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_train = train.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [9]:
df.columns

Index(['Unnamed: 0', 'archive_by_user', 'brand', 'cat1', 'cat2', 'cat3',
       'city', 'created_at', 'desc', 'id', 'image_count', 'mileage',
       'platform', 'price', 'title', 'type', 'year', 'cat1_2', 'cat1_2_3'],
      dtype='object')

In [0]:
X_test = test.apply(lambda i : i.title + ' ' + i.desc, axis=1)
Y_test = test.apply(lambda i : labels[i.cat1_2_3], axis=1)

In [11]:
Y_test

175660    41
397456    38
428870    17
611346    38
155346     3
105495    48
207640    40
535107    23
152350     2
634588    75
14941      3
709021     0
228674    13
694166     4
666666    38
755569     0
138343     3
189469     3
239775    56
249759     6
53961      3
41058     48
813160     3
261474     3
620991     3
532160    24
449691    27
451856    17
811355    17
156868     8
          ..
169164     3
812744     3
194769    19
709558    18
767447    37
56529     38
838757    60
389442    16
623795    18
175566    31
583925     3
70162     60
218937     0
182853    24
569116    48
109072    56
352330    68
429647     3
45132     48
21839     17
142241    25
178471    18
139082    56
857534     8
769382    24
464668    37
872089    70
321316    33
303723    17
16664     15
Length: 236909, dtype: int64

In [12]:
MAX_NB_WORDS = 440000

texts_train =X_train.astype(str)
texts_test = X_test.astype(str)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 438312 unique tokens.


In [0]:
index_to_word = dict((i, w) for w, i in tokenizer.word_index.items())

In [14]:
" ".join([index_to_word[i] for i in sequences[0]])

'قالیچه قالیچه فانتزی طرح گل'

In [0]:
from keras.preprocessing.sequence import pad_sequences

In [16]:
MAX_SEQUENCE_LENGTH =  200

x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)


Shape of data tensor: (710726, 200)
Shape of data test tensor: (236909, 200)


In [17]:


Y_train = to_categorical(np.asarray(Y_train))
Y_test = to_categorical(np.asarray(Y_test))
print('Shape of label tensor:', Y_train.shape)
print('Shape of label tensor:', Y_test.shape)


Shape of label tensor: (710726, 87)
Shape of label tensor: (236909, 87)


In [18]:
Y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [25]:
from keras.layers import Dense, Input, Flatten,Bidirectional
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100
N_CLASSES = 87

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

x = Bidirectional(LSTM(100))(embedded_sequences)
predictions = Dense(N_CLASSES, activation='softmax')(x)


model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 200, 100)          44000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_5 (Dense)              (None, 87)                17487     
Total params: 44,178,287
Trainable params: 44,178,287
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train, Y_train, validation_data=[x_test,Y_test],
          nb_epoch=10, batch_size=256)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 710726 samples, validate on 236909 samples
Epoch 1/10
710726/710726 [==============================] - 2675s 4ms/step - loss: 1.0561 - acc: 0.7438 - val_loss: 0.4922 - val_acc: 0.8753
Epoch 2/10
710726/710726 [==============================] - 2686s 4ms/step - loss: 0.3612 - acc: 0.9045 - val_loss: 0.3774 - val_acc: 0.8994
Epoch 3/10
710726/710726 [==============================] - 2692s 4ms/step - loss: 0.2402 - acc: 0.9323 - val_loss: 0.3607 - val_acc: 0.9027
Epoch 4/10
710726/710726 [==============================] - 2693s 4ms/step - loss: 0.1781 - acc: 0.9478 - val_loss: 0.3627 - val_acc: 0.9030
Epoch 5/10
 87296/710726 [==>...........................] - ETA: 35:25 - loss: 0.1273 - acc: 0.9618

In [0]:
from sklearn.metrics import roc_auc_score
output_test = model.predict(x_test)


In [0]:
from sklearn.metrics import classification_report
Y_test = np.argmax(Y_test,axis=1)
output_test = np.argmax(output_test,axis=1)
print(classification_report(Y_test, output_test))

In [0]:
tst = pd.read_csv('dataset.csv')

In [0]:
X_tst = tst.apply(lambda i : str(i.title) + ' ' + str(i.desc), axis=1)


In [0]:
texts_tst = X_tst.astype(str)


In [0]:
sequences_tst = tokenizer.texts_to_sequences(texts_tst)


In [0]:
MAX_SEQUENCE_LENGTH = 250

x_tst = pad_sequences(sequences_tst, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', x_tst.shape)

In [0]:
output_test = model.predict(x_tst)


In [0]:
pred=np.argmax(output_test,axis=1)

In [0]:
tst['cat1_2_3']=pred

In [0]:
merged = {}
for key, value in labels.items():
    merged[value] = key


In [0]:
pred.shape


In [0]:
tst['cat1_2_3'] = tst['cat1_2_3'].map(merged)

In [0]:
cat_tmp = tst['cat1_2_3'].str.split("|",n=1,expand=True)
tst['cat1'] = cat_tmp[0]
tst['cat2_3']=cat_tmp[1] 
cat2_tmp = tst['cat2_3'].str.split("|",n=1,expand=True)
tst['cat2'] = cat2_tmp[0]
tst['cat3']=cat2_tmp[1] 

In [0]:
tst[150000:150100]

In [0]:
tst = tst.drop(columns=['Unnamed: 0', 'archive_by_user', 'city', 'created_at', 'desc', 'id',
       'image_count', 'platform', 'price', 'title', 'cat1_2_3',
       'cat2_3',],axis=1)


In [0]:
tst.replace(['nan'], '', inplace=True)

In [0]:
tst

In [0]:
tst.to_csv("tst_deep2240.csv")